# Example of `InferenceData` schema in PyStan
The description of the `InferenceData` structure can be found in [GitHub](https://github.com/arviz-devs/arviz/blob/master/schema.md).

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html")

In [4]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,29200,12.0
Erika,27500,18.0


In [5]:
linreg_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
  real slack_comments[N];
}

parameters {
  real b0;
  real b1;
  real log_sigma;
}

transformed parameters {
  real<lower=0> sigma;
  sigma = exp(log_sigma);
}

model {
  b0 ~ normal(0,10);
  b1 ~ normal(0,10);
  for (i in 1:N) {
    slack_comments[i] ~ normal(b0 + b1 * time_since_joined[i], sigma);
  }
  
}

generated quantities {
    vector[N] log_lik;
    vector[N] slack_comments_hat;
    for (i in 1:N) {
        log_lik[i] = normal_lpdf(slack_comments[i] | b0 + b1 * time_since_joined[i], sigma);
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "time_since_joined": time_since_joined}

In [6]:
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_436a47b6d0471232cc823b9d65bd2bd6 NOW.


In [7]:
prior =  sm.sampling(data=linreg_data_dict, iter=200, chains=1, algorithm='Fixed_param', warmup=0)

In [8]:
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [9]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive="slack_comments_hat",
    prior_predictive="slack_comments_hat",
    observed_data=["slack_comments"],
    constant_data=["time_since_joined"],
    log_likelihood="log_lik",
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "log_lik": ["developer"],
        "slack_comments_hat": ["developer"],
        "time_since_joined": ["developer"],
    }
)

In [10]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> sample_stats_prior
	> prior_predictive
	> observed_data
	> constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [11]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:    (chain: 4, draw: 100)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0         (chain, draw) float64 12.18 -0.1928 -0.1928 ... 9.379 6.98 6.961
    b1         (chain, draw) float64 -3.278 6.105 6.105 ... -14.41 -17.81 -17.38
    log_sigma  (chain, draw) float64 10.46 9.441 9.441 ... 9.952 10.61 9.831
    sigma      (chain, draw) float64 3.487e+04 1.26e+04 ... 4.049e+04 1.861e+04
Attributes:
    created_at:                 2019-11-10T20:53:29.353030
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    accept_stat     (chain, draw) float64 0.8935 0.9752 0.6738 ... 0.4978 0.9424
    stepsize        (chain, draw) float64 0.3148 0.3148 0.3148 ... 0.2926 0.2926
    treedepth       (chain, draw) int64 6 6 1 5 5 2 7 7 7 ... 2 6 2 5 6 6 3 6 3
    n_leapfrog      (chain, draw) int64 79 87 3 43 63 3 ... 51 127 63 9 123 11
    diverging       (chain, draw) bool False False False ... False False False
    energy          (chain, draw) float64 55.57 55.7 54.71 ... 53.95 56.23 55.53
    log_likelihood  (chain, draw, developer) float64 -11.4 -11.42 ... -11.87
    lp              (chain, draw) float64 -53.96 -54.01 -54.01 ... -55.53 -54.01
Attributes:
    created_at:                 2019-11-10T20:53:29.358118
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int64 0 1 2 3
  * draw                (draw) int64 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 -3.492e+04 ... 1.574e+04
Attributes:
    created_at:                 2019-11-10T20:53:29.361049
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 29200 27500
Attributes:
    created_at:                 2019-11-10T20:53:29.345608
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2019-11-10T20:53:29.349936
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.prior

<xarray.Dataset>
Dimensions:    (chain: 1, developer: 5, draw: 200)
Coordinates:
  * chain      (chain) int64 0
  * draw       (draw) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 197 198 199
  * developer  (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    b0         (chain, draw) float64 0.9651 0.9651 0.9651 ... 0.9651 0.9651
    b1         (chain, draw) float64 -1.508 -1.508 -1.508 ... -1.508 -1.508
    log_sigma  (chain, draw) float64 -1.05 -1.05 -1.05 ... -1.05 -1.05 -1.05
    sigma      (chain, draw) float64 0.3501 0.3501 0.3501 ... 0.3501 0.3501
    log_lik    (chain, draw, developer) float64 -2.298e+08 ... -3.091e+09
Attributes:
    created_at:                 2019-11-10T20:53:29.362968
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [17]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 200)
Coordinates:
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    lp           (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2019-11-10T20:53:29.364991
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [18]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 200)
Coordinates:
  * chain               (chain) int64 0
  * draw                (draw) int64 0 1 2 3 4 5 6 ... 194 195 196 197 198 199
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 -5.736 ... -26.45
Attributes:
    created_at:                 2019-11-10T20:53:29.366357
    inference_library:          pystan
    inference_library_version:  2.19.1.1